# "2020 February One Day Holding Long Short Portfolio BackTest"
> "2020 February Portfolio"
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [long, short, longshort]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [31]:
#hide_input
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import gc
import glob
import matplotlib.pyplot as plt

In [32]:
#hide_input
#df_testp=pd.read_csv('D:\\Downloads\\20194dayreturnpred2019train60K.csv')
#df_test=pd.read_csv('D:\\Downloads\\sample28K20194dayreturnpred2019train60K.csv')
df_tes=pd.read_csv('D:\\Downloads\\10Q20204dayreturnpred2019train60K.csv')

In [33]:
#hide
#df_testp=df_testp.loc[df_testp.label!='nan'].loc[df_testp.Date>=20190101].loc[df_testp.Date<20190201].sort_values(by='Date',ascending=True).copy()
#df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20190101].loc[df_test.Date<20190201].sort_values(by='Date',ascending=True).copy()
df_tes=df_tes.loc[df_tes.label!='nan'].loc[df_tes.Date>=20200201].loc[df_tes.Date<20200301].sort_values(by='Date',ascending=True).copy()
#df_testp.reset_index(drop=True, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


**Long-Short Portfolio**

In [34]:
#hide
#df_agg=df_testp[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
#df_agg['cumreturn1']=0.0
#for l in range(len(df_agg)):
#    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])
    
#hide
#df_ag=df_test[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
#df_ag['cumreturn1a']=0.0
#for l in range(len(df_ag)):
#    df_ag['cumreturn1a'][l]=np.prod((1+(df_ag['labelr1']*df_ag['pred']/100))[:l+1])
    
df_a=df_tes[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [35]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [36]:
#hide_input
df_a

,Date,pred,labelr1,cumreturn1b,cumreturndow,cumreturnsp500
0,2020-02-03,-1.0,0.000000,1.000000,1.014980,1.014360
1,2020-02-03,1.0,-36.867945,0.631321,1.014980,1.014360
2,2020-02-04,-1.0,-4.597415,0.660345,1.026400,1.031375
3,2020-02-04,1.0,10.929751,0.732519,1.026400,1.031375
4,2020-02-05,-1.0,8.029553,0.673701,1.029813,1.034506
5,2020-02-05,1.0,2.184310,0.688417,1.029813,1.034506
6,2020-02-06,-1.0,2.643730,0.670217,1.024251,1.024743
7,2020-02-06,1.0,45.356286,0.974202,1.024251,1.024743
8,2020-02-07,-1.0,-7.641829,1.048649,1.031755,1.030881
9,2020-02-07,1.0,-2.437395,1.023090,1.031755,1.030881


In [37]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

#a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
#    x='Date:T',
#    y='cumreturn1a:Q',

#).properties(
#    width=1000,
#    height=1000
#)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3# + a4 + a5

alt.LayerChart(...)

**Long portfolio**

In [38]:
#hide
df_a=df_tes[['Date','pred','labelr1']].loc[df_tes.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [39]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [40]:
#hide_input
df_a

,Date,pred,labelr1,cumreturn1b,cumreturndow,cumreturnsp500
0,2020-02-03,1.0,-36.867945,0.631321,1.014980,1.014360
1,2020-02-04,1.0,10.929751,0.700322,1.026400,1.031375
2,2020-02-05,1.0,2.184310,0.715620,1.029813,1.034506
3,2020-02-06,1.0,45.356286,1.040198,1.024251,1.024743
4,2020-02-07,1.0,-2.437395,1.014844,1.031755,1.030881
5,2020-02-10,1.0,-5.124706,0.962836,1.033497,1.030864
6,2020-02-11,1.0,-2.926518,0.934659,1.040176,1.040550
7,2020-02-12,1.0,6.666667,0.996969,1.038480,1.036039
8,2020-02-13,1.0,-5.170124,0.945425,1.040395,1.035151
9,2020-02-14,1.0,0.000000,0.945425,1.037357,1.029309


In [41]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

#a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
#    x='Date:T',
#    y='cumreturn1a:Q',

#).properties(
#    width=1000,
#    height=1000
#)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3# + a4 + a5

alt.LayerChart(...)

**Short Portfolio**

In [42]:
#hide
    
df_a=df_tes[['Date','pred','labelr1']].loc[df_tes.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [43]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [44]:
#hide_input
df_a

,Date,pred,labelr1,cumreturn1b,cumreturndow,cumreturnsp500
0,2020-02-03,-1.0,0.000000,1.000000,1.014980,1.014360
1,2020-02-04,-1.0,-4.597415,1.045974,1.026400,1.031375
2,2020-02-05,-1.0,8.029553,0.961987,1.029813,1.034506
3,2020-02-06,-1.0,2.643730,0.936555,1.024251,1.024743
4,2020-02-07,-1.0,-7.641829,1.008125,1.031755,1.030881
5,2020-02-10,-1.0,-36.414232,1.375226,1.033497,1.030864
6,2020-02-11,-1.0,-4.128549,1.432002,1.040176,1.040550
7,2020-02-12,-1.0,-2.605301,1.469310,1.038480,1.036039
8,2020-02-13,-1.0,10.658952,1.312697,1.040395,1.035151
9,2020-02-14,-1.0,0.000000,1.312697,1.037357,1.029309


In [45]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

#a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
#    x='Date:T',
#    y='cumreturn1a:Q',

#).properties(
#    width=1000,
#    height=1000
#)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3# + a4 + a5

alt.LayerChart(...)